In [ ]:
import itertools
import random
import math
import matplotlib.pyplot as plt  # type: ignore
import geopandas as gpd  # type: ignore

In [ ]:
# Data from data.py
##Defining the towns
location_names = [
    "Windhoek", "Swakopmund", "Walvis Bay", "Otjiwarongo", "Tsumeb",
    "Grootfontein", "Mariental", "Keetmanshoop", "Ondangwa", "Oshakati"
]

In [ ]:
## Contains the distance data matrix for the towns.
distance_data = [
    [0, 361, 395, 249, 433, 459, 268, 497, 678, 712],
    [361, 0, 35.5, 379, 562, 589, 541, 859, 808, 779],
    [395, 35.5, 0, 413, 597, 623, 511, 732, 884, 855],
    [249, 379, 413, 0, 260, 183, 519, 768, 514, 485],
    [433, 562, 597, 260, 0, 60, 682, 921, 254, 288],
    [459, 589, 623, 183, 60, 0, 708, 947, 308, 342],
    [268, 541, 511, 519, 682, 708, 0, 231, 909, 981],
    [497, 859, 732, 768, 921, 947, 231, 0, 1175, 1210],
    [678, 808, 884, 514, 254, 308, 909, 1175, 0, 30],
    [712, 779, 855, 485, 288, 342, 981, 1210, 30, 0]
]

In [ ]:
# TSP class from tsp.py
## Creating the TSP class to store location_names, distances and total route distance.
class TSP:
    def __init__(self, location_names, distance_data):
        
        ##Initializing town names with their respective distance data.
        
        self.locations = location_names
        self.distances = distance_data

    def calculate_total_cost(self, path):
        
        
    ## Calculating total round-trip distance of a given path
        total_cost = 0
        for i in range(len(path)):
            origin = path[i]
            destination = path[(i + 1) % len(path)]  # loop back to start
            total_cost += self.distances[origin][destination]
        return total_cost

    def path_cost(self, path):
        
        ##Calculating  the total distance of a path.
        
        total = 0
        for i in range(len(path)):
            start = path[i]
            end = path[(i + 1) % len(path)]
            total += self.distances[start][end]
        return total

    def path_to_names(self, path):
        
        
        return [self.locations[i] for i in path]

In [ ]:
# SimulatedAnnealingEngine class from simulated_annealing.py
class SimulatedAnnealingEngine:
    def __init__(self, tsp_small, initial_temp=10000, cooling_rate=0.003, max_iter=10000):
        self.tsp = tsp_small
        self.temperature = initial_temp
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iter

    def generate_initial_path(self):
        ##Generating a random initial path
        path = list(range(len(self.tsp.locations)))
        random.shuffle(path)
        return path

    def swap_cities(self, path):
        ##Swapping two towns
        new_path = path[:]
        i, j = random.sample(range(len(new_path)), 2)
        new_path[i], new_path[j] = new_path[j], new_path[i]
        return new_path

    def acceptance_function(self, current_cost, new_cost, temperature):
        ##Probability function to accept/reject based on temperature
        if new_cost < current_cost:
            return 1.0
        return math.exp((current_cost - new_cost) / temperature)

    def cool_down(self, t):
        ## Implementing a cooling schedule (exponential decay)
        return t * (1 - self.cooling_rate)

    def solve(self):
        ##Running the simulated annealing algorithm
        current_route = self.generate_initial_path()
        current_distance = self.tsp.calculate_total_cost(current_route)

        best_route = current_route[:]
        best_distance = current_distance

        temperature = self.temperature
        iteration = 0

        while temperature > 1 and iteration < self.max_iterations:
            new_route = self.swap_cities(current_route)
            new_distance = self.tsp.calculate_total_cost(new_route)

            if self.acceptance_function(current_distance, new_distance, temperature) > random.random():
                current_route = new_route
                current_distance = new_distance

                if new_distance < best_distance:
                    best_route = new_route
                    best_distance = new_distance

            temperature = self.cool_down(temperature)
            iteration += 1
       ## Returns best route
        return best_route, best_distance

In [ ]:
# Functions and code from analysis.py
def brutel_force_tsp(tsp):
    best_route = None
    best_distance = float('inf')
    indices = list(range(len(tsp.locations)))

    for perm in itertools.permutations(indices[1:]): 
        route = [0] + list(perm)
        distance = tsp.path_cost(route)
        if distance < best_distance:
            best_distance = distance
            best_route = route
    return best_route, best_distance

In [ ]:
def run_analysis():
    # Using a subset of 4 towns to test and  keep brute force feasible
    locations_small_towns = ["Windhoek", "Swakopmund", "Walvis Bay", "Otjiwarongo"]
    small_distance_data = [
        [0, 361, 395, 249],      # Windhoek
        [361, 0, 35.5, 379],       # Swakopmund
        [395, 35.5, 0, 413],       # Walvis Bay
        [249, 379, 413, 0],      # Otjiwarongo
    ]

    tsp_small = TSP(locations_small_towns, small_distance_data)

    # Brute force algorithm
    brute_route, brute_distance = brutel_force_tsp(tsp_small)
    brute_named = [tsp_small.locations[i] for i in brute_route]
    print("Brute Force Best Route:", brute_named)
    print("Brute Force Distance:", brute_distance)

    # Simulated annealing algorithm
    solver = SimulatedAnnealingEngine(tsp_small, initial_temp=1000, cooling_rate=0.995, max_iter=5000)
    sa_route, sa_distance = solver.solve()
    sa_named = [tsp_small.locations[i] for i in sa_route]
    print("\nSimulated Annealing Route:", sa_named)
    print("Simulated Annealing Distance:", sa_distance)